In [ ]:
'''
Código para verificar se compensa realizar um parcelamento ou pagar à vista com
base no valor da compra.
'''
# Entrada da taxa de juros anual (aceitando ponto ou vírgula)
taxa_anual = input('Digite o valor da taxa anual de rendimento (%): ').replace(',', '.')
taxa_anual = float(taxa_anual) / 100  # Converte para decimal

# Calcula a taxa mensal com juros compostos
taxa_mensal = (1 + taxa_anual)**(1/12) - 1

# Entrada do valor inicial da compra (aceitando ponto ou vírgula)
valor_inicial = input('Digite o valor total da compra: R$').replace(',', '.')
valor_inicial = float(valor_inicial)  # Converte para decimal

# Verifica se há desconto à vista
r1 = input('Há desconto em pagar à vista? [S/N]: ').strip().upper()
while (r1 != 'S') and (r1 != 'N'):
    print('Resposta inválida! Tente novamente.')
    r1 = input('Há desconto em pagar à vista? [S/N]: ').strip().upper()
if r1 == 'S':
    desconto = float(input('Qual a porcentagem de desconto em pagar à vista? '))
    vista = valor_inicial - (valor_inicial * (desconto / 100))
else:
    vista = valor_inicial

# Verifica se há acréscimo em pagar parcelado 
# Compras no cartão de crédito são consideradas como 1 parcela
r2 = input('Há juros em pagar parcelado? [S/N]: ').strip().upper()
while (r2 != 'S') and (r2 != 'N'):
    print('Resposta inválida! Tente novamente.')
    r2 = input('Há juros em pagar parcelado? [S/N]: ').strip().upper()
if r2 == 'S':
    juros = float(input('Qual a porcentagem de juros em pagar parcelado? '))
    valor_inicial += valor_inicial * (juros / 100)

# Entrada do número de parcelas
qnt_parcelas = int(input('Digite o número de parcelas: '))
valor_parcela = valor_inicial / qnt_parcelas

# Simula os rendimentos com parcelas mensais
ganho = 0
total = valor_inicial
print(f'\n{"Parcelas":^12}{"Dívida":^10}{"Rendimento":^15}')  # Cabeçalho com centralização
print('-'*35)
for _ in range(qnt_parcelas):
    rendimento = total * taxa_mensal
    print(f'{_+1:^12}{total:>10.2f}{rendimento:>10.2f}')
    ganho += rendimento
    total -= valor_parcela

# Calcula o custo efetivo do parcelamento
valor_final_parcelado = valor_inicial - ganho

# Exibe os resultados
print(f'\nValor pagando à vista: R${vista:.2f}')
print(f'Rendimentos acumulados com o parcelamento: R${ganho:.2f}')
print(f'Valor pago parcelando (descontados os rendimentos): R${valor_final_parcelado:.2f}')

# Decide qual é mais vantajoso
if vista < valor_final_parcelado:
    print('\nCompensa pagar à vista.')
else:
    print('\nCompensa parcelar.')